In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
185,AFG,Asia,Afghanistan,2020-07-13,34451.0,0.0,1010.0,0.0,884.985,0.000,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
312,ALB,Europe,Albania,2020-07-13,3454.0,83.0,93.0,4.0,1200.222,28.841,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
503,DZA,Africa,Algeria,2020-07-13,19195.0,483.0,1011.0,7.0,437.732,11.015,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-12,France,171496.0,0.0,30027.0,0.0,0.0,0.0
0,2020-07-12,China,85134.0,0.0,4650.0,0.0,0.0,0.0
0,2020-07-12,Italy,243223.0,0.0,34981.0,0.0,0.0,0.0
0,2020-07-12,Spain,254769.0,0.0,28417.0,0.0,0.0,0.0
0,2020-07-12,United States,3303546.0,0.0,135740.0,0.0,0.0,0.0
0,2020-07-12,World,12924586.0,0.0,571021.0,0.0,0.0,0.0
0,2020-07-12,United Kingdom,289916.0,0.0,44902.0,0.0,0.0,0.0
0,2020-07-12,Germany,199258.0,0.0,9078.0,0.0,0.0,0.0
0,2020-07-12,Iran,258406.0,0.0,12771.0,0.0,0.0,0.0
0,2020-07-12,Turkey,212940.0,0.0,5365.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-13,France,171496.0,170752.0,30027.0,30004.0,744.0,23.0
0,2020-07-13,China,85134.0,85117.0,4650.0,4641.0,17.0,9.0
0,2020-07-13,Italy,243223.0,243061.0,34981.0,34954.0,162.0,27.0
0,2020-07-12,Spain,254769.0,253908.0,28417.0,28403.0,861.0,14.0
0,2020-07-13,United States,3303546.0,3304942.0,135740.0,135205.0,-1396.0,535.0
0,2020-07-13,World,12924586.0,12888021.0,571021.0,568736.0,36565.0,2285.0
0,2020-07-13,United Kingdom,289916.0,289603.0,44902.0,44819.0,313.0,83.0
0,2020-07-13,Germany,199258.0,198963.0,9078.0,9064.0,295.0,14.0
0,2020-07-13,Iran,258406.0,257303.0,12771.0,12829.0,1103.0,-58.0
0,2020-07-13,Turkey,212940.0,212993.0,5365.0,5363.0,-53.0,2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")